In [14]:
# dependencies
import json
import pandas as pd
import numpy as np
import os

In [23]:
# import and open wikipedia JSON

# file path
file_dir = "C://Users/tyler/Desktop/dabc-assignments/movies-ETL"


# open file
with open('resources/wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)

In [24]:
# check length
len(wiki_movies_raw)

7311

In [25]:
# inspect first 5 records
wiki_movies_raw[:5]

[{'url': 'https://en.wikipedia.org/wiki/The_Adventures_of_Ford_Fairlane',
  'year': 1990,
  'imdb_link': 'https://www.imdb.com/title/tt0098987/',
  'title': 'The Adventures of Ford Fairlane',
  'Directed by': 'Renny Harlin',
  'Produced by': ['Steve Perry', 'Joel Silver'],
  'Screenplay by': ['David Arnott', 'James Cappe', 'Daniel Waters'],
  'Story by': ['David Arnott', 'James Cappe'],
  'Based on': ['Characters', 'by Rex Weiner'],
  'Starring': ['Andrew Dice Clay',
   'Wayne Newton',
   'Priscilla Presley',
   'Lauren Holly',
   'Morris Day',
   'Robert Englund',
   "Ed O'Neill"],
  'Narrated by': 'Andrew "Dice" Clay',
  'Music by': ['Cliff Eidelman', 'Yello'],
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Michael Tronick',
  'Productioncompany ': 'Silver Pictures',
  'Distributed by': '20th Century Fox',
  'Release date': ['July 11, 1990', '(', '1990-07-11', ')'],
  'Running time': '102 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$20 million',


In [21]:
# inspect the last 5 records
wiki_movies_raw[-5:]

[{'url': 'https://en.wikipedia.org/wiki/Holmes_%26_Watson',
  'year': 2018,
  'imdb_link': 'https://www.imdb.com/title/tt1255919/',
  'title': 'Holmes & Watson',
  'Directed by': 'Etan Cohen',
  'Produced by': ['Will Ferrell',
   'Adam McKay',
   'Jimmy Miller',
   'Clayton Townsend'],
  'Screenplay by': 'Etan Cohen',
  'Based on': ['Sherlock Holmes',
   'and',
   'Dr. Watson',
   'by',
   'Sir Arthur Conan Doyle'],
  'Starring': ['Will Ferrell',
   'John C. Reilly',
   'Rebecca Hall',
   'Rob Brydon',
   'Steve Coogan',
   'Ralph Fiennes'],
  'Music by': 'Mark Mothersbaugh',
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Dean Zimmerman',
  'Productioncompanies ': ['Columbia Pictures',
   'Gary Sanchez Productions',
   'Mosaic Media Group',
   'Mimran Schur Pictures'],
  'Distributed by': 'Sony Pictures Releasing',
  'Release date': ['December 25, 2018',
   '(',
   '2018-12-25',
   ')',
   '(United States)'],
  'Running time': '90 minutes',
  'Country': 'United States',
  'Language

In [26]:
# check some records in the middle
wiki_movies_raw[3600:3605]

[{'url': 'https://en.wikipedia.org/wiki/Benji:_Off_the_Leash!',
  'year': 2004,
  'imdb_link': 'https://www.imdb.com/title/tt0315273/',
  'title': 'Benji: Off the Leash!',
  'Directed by': 'Joe Camp',
  'Written by': 'Joe Camp',
  'Starring': ['Benji', 'Nick Whitaker', 'Shaggy', 'Gypsy the Cockatoo'],
  'Music by': 'Antonio di Lorenzo',
  'Productioncompany ': 'Mulberry Square Productions',
  'Distributed by': 'Mulberry Square Productions',
  'Release date': ['March 26, 2004', '(', '2004-03-26', ')'],
  'Running time': '97 min',
  'Country': 'United States',
  'Language': 'English',
  'Box office': '$3,817,362'},
 {'url': 'https://en.wikipedia.org/wiki/The_Best_Thief_in_the_World',
  'year': 2004,
  'imdb_link': 'https://www.imdb.com/title/tt0389796/',
  'title': 'The Best Thief in the World',
  'Directed by': 'Jacob Kornbluth',
  'Produced by': ['Tim Perrell', 'Nicola Usborne'],
  'Written by': 'Jacob Kornbluth',
  'Starring': ['Marc Rozendaal',
   'Michael Silverman',
   'David Warsh

In [27]:
# read csv from the kaggle dataset
kaggle_metadata = pd.read_csv("resources/movies_metadata.csv", low_memory=False)
ratings = pd.read_csv("resources/ratings.csv")

In [29]:
kaggle_metadata.sample(n=5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
17991,False,NaN,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",NaN,37079,tt0051666,en,God's Little Acre,"In the 1950s, a poor Georgia cotton farmer and...",...,1958-09-23,0.0,118.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"Love! Hate! Pride! Passion! Rampant, Riotous I...",God's Little Acre,False,6.6,4.0
24795,False,NaN,0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",NaN,73103,tt0479008,en,Dante's Inferno,A darkly comedic travelogue of the underworld ...,...,2007-01-01,0.0,88.0,[],Released,NaN,Dante's Inferno,False,5.7,3.0
33813,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",http://www.eunwe-movie.kr/index.htm,199584,tt2967578,ko,은밀하게 위대하게,"As a North Korean sleeper cell agent, Ryu-han ...",...,2013-06-05,0.0,124.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,"Secretly, Greatly",False,7.0,53.0
31746,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,34334,tt0105572,en,That Night,A coming-of-age story about an eleven-year-old...,...,1992-12-17,0.0,89.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,In everyone's life there's a friendship you ne...,That Night,False,6.3,15.0
8517,False,NaN,70000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 27, 'na...",NaN,29071,tt0051128,en,The Undead,Two psychics send a woman back in time to find...,...,1957-03-01,0.0,71.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror... that screams from the grave!,The Undead,False,4.4,7.0


In [30]:
ratings.sample(n=5)

,userId,movieId,rating,timestamp
8141688,83917,3441,3.0,1236375820
19133576,198746,7361,5.0,1178805911
20294107,210880,1080,3.0,951704237
20211250,210119,5959,0.5,1231537684
24373382,253135,44191,4.5,1199658191


In [35]:
# convert wiki_movies_raw to a dataframe
wiki_movies_df = pd.DataFrame(wiki_movies_raw)

In [37]:
# using list comprehension to clean the data
# filter for rows that have a director listed under imdb_link
# filter for rows that have "no of episodes" to get rid of TV shows
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]
len(wiki_movies)

7076

In [44]:
# create a funciton to help clean the data
    # step 1 = make an empty dictionary to hold the alternative titles
    # step 2 = loop through a list of all alternative title keys
    # step 2a = check if the currenty key exists in the movie object
    # step 2b = if true, remove the key-value pair and add to the alt_titles dictionary
    # step 3 = after looping through all keys, add alt_titles dict to the movie object
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    return movie

In [45]:
#Create a list of cleaned movies with a list comprehension
clean_movies = [clean_movie(movie) for movie in wiki_movies]

#Set wiki_movies_df to be the data frame created from clean_movies and print out a list of the columns
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

['Adaptation by',
 'Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Length',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Picture format',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Running time',
 'Screen story by',
 'Screenplay by',
 'Starring',
 'Story by',
 'Suggested by',
 'Theme music composer',
 'Venue',
 'Voices of',
 'Written by',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']